In [1]:
from pymatgen.io.vasp import Poscar, Xdatcar
from pymatgen.symmetry.groups import SpaceGroup
import numpy as np
from site_analysis import Atom, Analysis, PolyhedralSite
from site_analysis.tools import get_nearest_neighbour_indices
from collections import Counter
import tqdm
import matplotlib.pyplot as plt
import yaml

import sys
sys.path.insert(0, "../../scripts/")
from utils import flatten_list, get_structures, indices_by_species

from scipy.stats import gaussian_kde
from scipy.ndimage.filters import gaussian_filter1d

In [2]:
directory = '../../data'
structure = Poscar.from_file(f'{directory}/reference_structures/Li6PS5I_alltet_sites.POSCAR.vasp').structure

In [3]:
from pymatgen import Structure
lattice = structure.lattice
t0 = Structure.from_spacegroup(sg='F-43m', lattice=lattice, species=['P'], coords=[[0.5, 0.0, 0.0]])
t1 = Structure.from_spacegroup(sg='F-43m', lattice=lattice, species=['Li'], coords=[[0.9, 0.9, 0.6]])
t2 = Structure.from_spacegroup(sg='F-43m', lattice=lattice, species=['Li'], coords=[[0.23, 0.92, 0.08]])
t3 = Structure.from_spacegroup(sg='F-43m', lattice=lattice, species=['Li'], coords=[[0.25, 0.25, 0.25]])
t4 = Structure.from_spacegroup(sg='F-43m', lattice=lattice, species=['Li'], coords=[[0.15, 0.15, 0.15]])
t5 = Structure.from_spacegroup(sg='F-43m', lattice=lattice, species=['Li'], coords=[[0.0, 0.183, 0.183]])
tet_sites = [t0, t1, t2, t3, t4, t5]

In [77]:
def tetrahedral_site_analysis( structures, x_spec ):
    md_structure = structures[0]
    
    Atom.reset_index()
    atoms = [Atom(species_string='Li') for site in md_structure if site.species_string is 'Li']

    t0_indices = get_nearest_neighbour_indices( md_structure, t0, vertex_species=['S', x_spec], n_coord=4)
    t1_indices = get_nearest_neighbour_indices( md_structure, t1, vertex_species=['S', x_spec], n_coord=4)
    t2_indices = get_nearest_neighbour_indices( md_structure, t2, vertex_species=['S', x_spec], n_coord=4)
    t3_indices = get_nearest_neighbour_indices( md_structure, t3, vertex_species=['S', x_spec], n_coord=4)
    t4_indices = get_nearest_neighbour_indices( md_structure, t4, vertex_species=['S', x_spec], n_coord=4)
    t5_indices = get_nearest_neighbour_indices( md_structure, t5, vertex_species=['S', x_spec], n_coord=4)
    
    PolyhedralSite.reset_index()

    t0_sites = [ PolyhedralSite(vertex_species=['S', x_spec], vertex_indices=vi, 
                 label='0') for vi in t0_indices ]
    t1_sites = [ PolyhedralSite(vertex_species=['S', x_spec], vertex_indices=vi, 
                 label='1') for vi in t1_indices ]
    t2_sites = [ PolyhedralSite(vertex_species=['S', x_spec], vertex_indices=vi, 
                 label='2') for vi in t2_indices ]
    t3_sites = [ PolyhedralSite(vertex_species=['S', x_spec], vertex_indices=vi, 
                 label='3') for vi in t3_indices ]
    t4_sites = [ PolyhedralSite(vertex_species=['S', x_spec], vertex_indices=vi, 
                 label='4') for vi in t4_indices ]
    t5_sites = [ PolyhedralSite(vertex_species=['S', x_spec], vertex_indices=vi, 
                 label='5') for vi in t5_indices ]
    sites = t0_sites + t1_sites + t2_sites + t3_sites + t4_sites + t5_sites

    analysis = Analysis(atoms=atoms, sites=sites)
    analysis.trajectory_from_structures( structures, progress='notebook')
    return analysis

def site_populations(analysis):
    c = Counter()
    for site in analysis.sites:
        c[site.label] += len([ 1 for ts in site.trajectory if len(ts)>0 ])
    total = sum(c.values())
    return {label: n/total for label, n in c.items()}

In [4]:
with open('../md_runs.yaml', 'r') as f:
    md_runs = yaml.load(f)
print(md_runs)

{'Li6PS5I': {'0p': [1, 2, 3, 4, 5, 6], '50p': [1, 2, 3, 4, 5, 6, 7], '100p': [1, 2, 3, 4, 5, 6, 7]}, 'Li6PS5Cl': {'0p': [1, 2, 3, 4, 5], '50p': [1, 2, 3, 4, 5, 6], '100p': [1, 2, 3, 4, 5, 6]}}


In [6]:
data_dir = '../../data'

x_spec = {'Li6PS5I': 'I', 'Li6PS5Cl': 'Cl'}

analysis = {}
lattice = {}
for system in md_runs:
    analysis[system] = {}
    lattice[system] = {}
    for disorder, runs in md_runs[system].items():
        xdatcar_filenames = [ f'{data_dir}/{system}/{disorder}/run{i}/inherent_XDATCAR.gz' for i in runs ]
        xdatcars = ( Xdatcar( f ) for f in xdatcar_filenames )
        structures = flatten_list( [ x.structures for x in xdatcars ] )
        break

In [28]:
md_structure = structures[0]

Atom.reset_index()
atoms = [Atom(species_string='Li') for site in md_structure if site.species_string is 'Li']

In [29]:
PolyhedralSite.reset_index()

In [30]:
from site_analysis.shortest_distance_site import ShortestDistanceSite

In [36]:
t0_sites = [ ShortestDistanceSite(frac_coords=c, label='0') for c in t0 ]
t1_sites = [ ShortestDistanceSite(frac_coords=c, label='1') for c in t1 ]
t2_sites = [ ShortestDistanceSite(frac_coords=c, label='2') for c in t2 ]
t3_sites = [ ShortestDistanceSite(frac_coords=c, label='3') for c in t3 ]
t4_sites = [ ShortestDistanceSite(frac_coords=c, label='4') for c in t4 ]
t5_sites = [ ShortestDistanceSite(frac_coords=c, label='5') for c in t5 ]

sites = t0_sites + t1_sites + t2_sites + t3_sites + t4_sites + t5_sites

In [37]:
analysis = Analysis(atoms=atoms, sites=sites)
analysis.trajectory_from_structures( structures, progress='notebook')

ValueError: shapes (1,136) and (3,3) not aligned: 136 (dim 1) != 3 (dim 0)

In [34]:
t0_sites

In [13]:
md_structure = structures[0]

Atom.reset_index()
atoms = [Atom(species_string='Li') for site in md_structure if site.species_string is 'Li']

t0_indices = get_nearest_neighbour_indices( md_structure, t0*[2,2,2], vertex_species=['S', x_spec], n_coord=4)
t1_indices = get_nearest_neighbour_indices( md_structure, t1*[2,2,2], vertex_species=['S', x_spec], n_coord=4)
t2_indices = get_nearest_neighbour_indices( md_structure, t2*[2,2,2], vertex_species=['S', x_spec], n_coord=4)
t3_indices = get_nearest_neighbour_indices( md_structure, t3*[2,2,2], vertex_species=['S', x_spec], n_coord=4)
t4_indices = get_nearest_neighbour_indices( md_structure, t4*[2,2,2], vertex_species=['S', x_spec], n_coord=4)
t5_indices = get_nearest_neighbour_indices( md_structure, t5*[2,2,2], vertex_species=['S', x_spec], n_coord=4)

PolyhedralSite.reset_index()

t0_sites = [ PolyhedralSite(vertex_species=['S', x_spec], vertex_indices=vi, 
             label='0') for vi in t0_indices ]
t1_sites = [ PolyhedralSite(vertex_species=['S', x_spec], vertex_indices=vi, 
             label='1') for vi in t1_indices ]
t2_sites = [ PolyhedralSite(vertex_species=['S', x_spec], vertex_indices=vi, 
             label='2') for vi in t2_indices ]
t3_sites = [ PolyhedralSite(vertex_species=['S', x_spec], vertex_indices=vi, 
             label='3') for vi in t3_indices ]
t4_sites = [ PolyhedralSite(vertex_species=['S', x_spec], vertex_indices=vi, 
             label='4') for vi in t4_indices ]
t5_sites = [ PolyhedralSite(vertex_species=['S', x_spec], vertex_indices=vi, 
             label='5') for vi in t5_indices ]
sites = t0_sites + t1_sites + t2_sites + t3_sites + t4_sites + t5_sites

In [27]:
# sites are grouped in equivalent sets of 8
for vi in t0_indices:
    print( [md_structure[i].frac_coords for i in vi ])

[array([0.18880848, 0.05523986, 0.06139044]), array([0.31010476, 0.05418107, 0.94771186]), array([0.18692827, 0.93952284, 0.94496245]), array([0.30542985, 0.94188803, 0.06471598])]
[array([0.18702811, 0.05978181, 0.56071977]), array([0.30528239, 0.05574242, 0.44446376]), array([0.1922343 , 0.93657963, 0.44595785]), array([0.29953233, 0.93846156, 0.56439322])]
[array([0.18944828, 0.5603338 , 0.05344615]), array([0.3064757 , 0.55618103, 0.93829128]), array([0.19016999, 0.43862062, 0.94546362]), array([0.31097055, 0.44704559, 0.06238329])]
[array([0.19267563, 0.55870017, 0.56356581]), array([0.30694168, 0.56005162, 0.44476412]), array([0.18749274, 0.44515534, 0.4441005 ]), array([0.30910344, 0.43894409, 0.55691485])]
[array([0.68675428, 0.05341701, 0.05685099]), array([0.80607909, 0.05447409, 0.93665898]), array([0.68576905, 0.93852288, 0.9440404 ]), array([0.81030653, 0.94014251, 0.0515226 ])]
[array([0.69197271, 0.0562448 , 0.56695137]), array([0.80454605, 0.05695627, 0.4451205 ]), arra

In [22]:
s.vertex_coords

In [70]:
li_coords = [ s.frac_coords for struct in structures for s in struct if s.species_string is 'Li' ]

In [71]:
len(li_coords)

134400

In [72]:
proj_coords = [ list(c) for c in np.mod(li_coords*np.array([2,2,2]), 1.0) ]

In [73]:
proj_coords

[[0.1923447, 0.46393244, 0.69674328],
 [0.31746934, 0.02481774, 0.69962696],
 [0.25483538, 0.4202157799999999, 0.9156243],
 [0.01927032, 0.1999564599999999, 0.780349],
 [0.47355056000000006, 0.18040094, 0.69682824],
 [0.2047547999999999, 0.2836413, 0.5046137799999999],
 [0.198793, 0.29420578, 0.53251048],
 [0.31269718, 0.0228684400000001, 0.7056813200000001],
 [0.33522676, 0.30377034, 0.96033446],
 [0.21234216, 0.2106936, 0.9854188800000001],
 [0.19356262, 0.047767820000000016, 0.8212769],
 [0.47675864, 0.19522674000000007, 0.6839580999999999],
 [0.03188612000000002, 0.21336586, 0.82203204],
 [0.21222384000000005, 0.48106056, 0.6975387399999999],
 [0.19679248000000005, 0.02592024000000004, 0.83589486],
 [0.20881864000000006, 0.4757962200000001, 0.7397707],
 [0.22002852, 0.00547936, 0.77362068],
 [0.22408856, 0.2681315, 0.5314108],
 [0.30478936, 0.1561281000000001, 0.53660894],
 [0.2505205, 0.10673982000000004, 0.5619223600000001],
 [0.33548727999999994, 0.29798792, 0.9871576],
 [0.0395

In [74]:
lattice = structure.lattice
sym_struct = Structure.from_spacegroup(sg='F-43m', lattice=lattice, species=['Li']*len(proj_coords), coords=proj_coords)

In [75]:
len(sym_struct)

12900912

In [78]:
tetrahedral_site_analysis([sym_struct], x_spec='S')

KeyboardInterrupt: 

192